# Participating in a Competition

<div class="admonition abstract highlight">
    <p class="admonition-title">In short</p>
    <p>This tutorial walks you through how to fetch an active competition from Polaris, prepare your predictions and then submit them for secure evaluation by the Polaris Hub.</p>
</div>

Participating in a competition on Polaris is very similar to participating in a standard benchmark. The main difference lies in how predictions are prepared and how they are evaluated. We'll touch on each of these topics later in the tutorial. 

Before continuing, please ensure you are logged into Polaris.

In [8]:
# Note: Cell is tagged to not show up in the mkdocs build
%load_ext autoreload
%autoreload 2

In [ ]:
import polaris as po
from polaris.hub.client import PolarisHubClient

# Don't forget to add your Polaris Hub username below!
MY_POLARIS_USERNAME = ""

client = PolarisHubClient()
client.login()

## Fetching a Competition

As with standard benchmarks, Polaris provides simple APIs that allow you to quickly fetch a competition from the Polaris Hub. All you need is the unique identifier for the competition which follows the format of `competition_owner`/`competition_name`.

In [10]:
competition_id = "polaris/hello-world-competition"
competition = po.load_competition(competition_id)

## Participate in the Competition
The Polaris library is designed to make it easy to participate in a competition. In just a few lines of code, we can get the train and test partition, access the associated data in various ways and evaluate our predictions. There's two main API endpoints. 

- `get_train_test_split()`: For creating objects through which we can access the different dataset partitions.
- `submit_predictions()`: For submitting the predictions to an active competition.

In [ ]:
train, test = competition.get_train_test_split()

Similar to benchmarks, the created test and train objects support various flavours to access the data.

In [ ]:
# The objects are iterable
for x, y in train:
    pass

# The objects can be indexed
for i in range(len(train)):
    x, y = train[i]

# The objects have properties to access all data at once. Use this with
# caution if the underlying dataset is large!
x = train.inputs
y = train.targets

Now, let's create some predictions against the imaginary `hello-world-competition`. Let's assume we train a simple random forest model on the ECFP representation through scikit-learn and datamol, and then we submit our results for secure evaluation by the Polaris Hub.

In [ ]:
import datamol as dm
from sklearn.ensemble import RandomForestRegressor

# Load the competition (automatically loads the underlying dataset as well)
competition = po.load_competition("polaris/hello-world-benchmark")

# Get the split and convert SMILES to ECFP fingerprints by specifying a featurize function.
train, test = competition.get_train_test_split(featurization_fn=dm.to_fp)

# Define a model and train
model = RandomForestRegressor(max_depth=2, random_state=0)
model.fit(train.X, train.y)

predictions = model.predict(test.X)

Now that we have created some predictions, we can construct a `CompetitionPredictions` object that will prepare our predictions for evaluation by the Polaris Hub. Here, you can also add metadata to your predictions to better describe your results and how you achieved them. 

In [ ]:
from polaris.evaluate import CompetitionPredictions

competition_predictions = CompetitionPredictions(
    name="hello-world-result",
    predictions=predictions,
    target_labels=competition.target_cols,
    test_set_labels=competition.test_set_labels,
    test_set_sizes=competition.test_set_sizes,
    github_url="https://github.com/polaris-hub/polaris-hub",
    paper_url="https://polarishub.io/",
    description="Hello, World!",
)

Once your `CompetitionPredictions` object is created, you're ready to submit them for evaluation! This will automatically save your result to the Polaris Hub, but it will be private until the competition closes.

In [ ]:
results = competition.evaluate(competition_predictions)

client.close()

That's it! Just like that you have partaken in your first Polaris competition. Keep an eye on that leaderboard when it goes public and best of luck in your future competitions!

The End.

---